<a href="https://colab.research.google.com/github/Sereniiti/models-exploration/blob/develop/Ayman/08_Bert_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Seneriiti with Bert** 

### Dataset:
The dataset we will contains sentences with 7 categories (from very violent to excellent).

**Summary:**
1. !pip install transformers
2. import data set
>- check and clean data set from Null values converting to 0 or removing the records.
>- check and remove doyplicated sentences from dataset.
>- map category column records from text to numbers.

3. Loading the Pre-trained BERT model DistilBERT or BERT importing import transformers
4. Load pretrained model/tokenizer
5. Preparing the Dataset before using with bert (Tokenization, Padding defining the number of records we are going to train and test, Masking)
6. create an input tensor out of the padded token matrix, and send that to DistilBERT
7. split our datset into a training set and testing set (from sklearn.model_selection import train_test_split)
8. train the LogisticRegression model
9. use confusion_matrix to figure out how is our training accuracy, we can use also (pd.crosstab(test_labels, predict_md, rownames=['True'], colnames=['Predicted'], margins=True))
10. predict test input text


In [5]:
# Install transformers library which provides us with an implementation of DistilBERT as well as pretrained versions of the model.
!pip install transformers -q gwpy &> /dev/null

In [6]:
# to mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Hide warnings if there are any
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/content/drive/MyDrive/sereniiti_project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing and prepare dataset

In [7]:
# import pandas
import pandas as pd

# import the csv file in df data frame
df = pd.read_csv('/content/drive/MyDrive/sereniiti_project/sereniiti_db.csv',sep=';',index_col='main_id_pk')

In [8]:
import smodel

In [9]:
#preprocessing dataset sentences.
list=[]
for item in df['text_en']:
  list.append(smodel.text_preprocessing(item))
df['words']=list

In [10]:
#encode category variable into numeric
df.step_id_fk= pd.Categorical(df.step_id_fk)
df['step_code'] = df.step_id_fk.cat.codes
df=df[['words','rating_id_fk','step_id_fk','step_code']]

In [11]:
smodel.bert_model_mcat('light', pd, df['words'][:3000], df['rating_id_fk'][:3000],df['step_code'][:3000],'train',None )


Transformation: creating model instance ... steps



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



-------------------Transformation Done -------------------
Transform column to tokens words then numbers 
----------------------------------

by tokenized firs 3 records like this : 

 main_id_pk
1    [101, 3438, 2199, 18047, 2015, 1998, 2017, 218...
2    [101, 1037, 2297, 2817, 8920, 1996, 3896, 1997...
3    [101, 1037, 2502, 3124, 2066, 2017, 3791, 2000...
Name: words, dtype: object


Berting padding and attention mask ... step 
----------------------------------

 padding ... step 
----------------------

padded shape:  (3000, 108)

 masking ... step 
----------------------

attention_mask shape:  (3000, 108)

tensoring ... step 
----------------------

input_ids 
 ----------------------
 tensor([[ 101, 3438, 2199,  ...,    0,    0,    0],
        [ 101, 1037, 2297,  ...,    0,    0,    0],
        [ 101, 1037, 2502,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2031,  ...,    0,    0,    0],
        [ 101, 1045, 2031,  ...,    0,    0,    0],
        [ 101, 1045, 2031

In [12]:
def predict_con_multiline(txt):
  x_new_df=pd.DataFrame({'token':txt.split(",")})
  
  print ('\nyour inquiry for : \n', x_new_df['token'],'\n\n...... please wait..., it will take some time\n' )
  smodel.bert_model_mcat('light', pd, df['words'][:3000], df['rating_id_fk'][:3000],df['step_code'][:3000],'test',x_new_df['token'] )
  x_new_df['predicting']= smodel.predict_md_test
  x_new_df['ratings']=x_new_df['predicting'].map({0:'very likely to elicit a defensive reaction or high pain in other people', 1:'likely to elicit a defensive reaction in other people',2:'slight tendance to elicit defensiveness in other people', 3:'slight tendance to encourage dialogue',4:'likely to encourage an open dialogue or neutral',5:'very likely to encourage an open dialogue which is mutually satisfying'})

  x_new_df['predicting_step']= smodel.predict_md_test2
  x_new_df['classification']=x_new_df['predicting_step'].map({0:'Facts, needs',1:'emotions',2:'emotions, needs',3:'emotions, propositions',4:'facts',5:'facts, emotions',6:'facts, emotions, needs',7:'facts, emotions, needs, propositions',8:'facts, emotions, propositions',9:'facts, feelings  ',10:'facts, intentions',11:'facts, needs',12:'facts, needs, feelings', 13:'facts, needs, propositions',14:'facts, propositions',15:'facts, propositions, intentions',16:'feelings',17:'intentions',18:'needs',19:'propositions',20:'propositions, feelings',21:'propositions, intentions',22:'propositions, needs',23:'requests' })  
  x_new_df['result']='\n'+x_new_df['token']+' \n**rating as: '+ x_new_df['ratings'] +' \n**Classified as: '+ x_new_df['classification'] +'\n'

  print ('\nResult abelow: \n', '====================\n')
  for item in x_new_df['result']:
    print(item)

In [13]:
# data entry test model
#========================
#input the data for testing the model
X_new = input('please input your text to test : ')

predict_con_multiline(X_new)

please input your text to test : You inspire me., You listen to me and you shut up, You make my heart warm., You make me ashamed, go hide.

your inquiry for : 
 0                      You inspire me.
1     You listen to me and you shut up
2              You make my heart warm.
3                  You make me ashamed
4                             go hide.
Name: token, dtype: object 

...... please wait..., it will take some time

Transform column to tokens words then numbers 
----------------------------------

by tokenized firs 3 records like this : 

 0                  [101, 2017, 18708, 2033, 1012, 102]
1    [101, 2017, 4952, 2000, 2033, 1998, 2017, 3844...
2       [101, 2017, 2191, 2026, 2540, 4010, 1012, 102]
Name: token, dtype: object


 padding ... step 
----------------------

padded shape:  (5, 108)

 masking ... step 
----------------------

attention_mask shape:  (5, 108)

input_ids 
 ----------------------
 tensor([[  101,  2017, 18708,  2033,  1012,   102,     0,     0,    